In [1]:
import numpy as np
import pandas as pd
import keras
import eng_to_ipa as ipa

Using TensorFlow backend.


In [37]:
df = pd.read_csv('Multiple_Forms.csv', index_col = 0)
df.head()

,Author,Poem,Form
0,William Shakespeare,"From fairest creatures we desire increase,\r\n...",Sonnet
1,William Shakespeare,"When forty winters shall besiege thy brow,\r\n...",Sonnet
2,William Shakespeare,Look in thy glass and tell the face thou viewe...,Sonnet
3,William Shakespeare,"Unthrifty loveliness, why dost thou spend\r\n ...",Sonnet
4,William Shakespeare,"Those hours, that with gentle work did frame\r...",Sonnet


In [39]:
poems = df['Poem'][df.Form == 'Sonnet']
poems

0     From fairest creatures we desire increase,\r\n...
1     When forty winters shall besiege thy brow,\r\n...
2     Look in thy glass and tell the face thou viewe...
3     Unthrifty loveliness, why dost thou spend\r\n ...
4     Those hours, that with gentle work did frame\r...
                            ...                        
95    Whate'er the cost to me, with this farewell,\n...
96    If the past year were offered me again,\nWith ...
97    Nay, dear one, ask me not to leave thee yet.\n...
98    Where is the pride for which I once was blamed...
99    I sue thee not for pity on my case.\nIf I have...
Name: Poem, Length: 348, dtype: object

In [40]:
text = ''
for poem in poems:
    text += str(poem) + '\n'
len(text)

258640

In [44]:
text = text.replace("’", "'")
text = text.replace("‘", "'")
text = text.replace("“", " ")
text = text.replace("'d", "ed")
text = text.replace("(", "")
text = text.replace(")", "")
text = text.replace(" '", " ")
text = text.replace("' ", " ")
text = text.replace('"', '')
text = text.replace("--", " ")
text = text.replace(":-", " ")
text = text.replace("-:", " ")
text = text.replace(".-", " ")
text = text.replace("- ", " ")
text = text.replace(" -", " ")
text = text.replace(" –", " ")
while text.find(" . ") != -1:
    text = text.replace(" . ", " ")
while text.find("..") != -1:
    text = text.replace("..", ".")
text = text.replace("?.", "? ")
text = text.replace("!.", "! ")
text = text.replace("!-", "! ")
text = text.replace(".", " ")
text = text.replace(",", " ")

In [45]:
word_list = text.split()
a = 0
roman_num = ['ii','iii','iv','v','vi','vii','viii','ix','x','xi','xii','xiii','xiv','xv','xvi','xvii',
             'xviii','xix','xx','xxi','xxii','xxiii','xxiv','xxv','xxvi','xxvii','xxviii','xxix','xxx']
for i in range(len(word_list) - a):
    word_list[i - a] = word_list[i - a].lower().strip("“”''-.?:;!,[]~`’—&")
    if word_list[i - a] in ['nan','','*'] or word_list[i-a][0] == '&' or word_list[i-a].isdigit() \
     or word_list[i-a] in roman_num:
        word_list.pop(i - a)
        a += 1
len(word_list)

46156

In [46]:
print(word_list)

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', "beauty's", 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', "feed'st", 'thy', "light's", 'flame', 'with', 'self-substantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thyself', 'thy', 'foe', 'to', 'thy', 'sweet', 'self', 'too', 'cruel', 'thou', 'that', 'art', 'now', 'the', "world's", 'fresh', 'ornament', 'and', 'only', 'herald', 'to', 'the', 'gaudy', 'spring', 'within', 'thine', 'own', 'bud', 'buriest', 'thy', 'content', 'and', 'tender', 'churl', "mak'st", 'waste', 'in', 'niggarding', 'pity', 'the', 'world', 'or', 'else', 'this', 'glutton', 'be', 'to', 'eat', 'the', "world's", 'due', 'by', 'the', 'grave', 'and', 'thee', 'when', 'forty', 'winters', 'shall', 'besiege', 'thy', 'brow', 'and', 'dig', 'deep', 'trench

In [47]:
maxlen = 6                                                            
step = 1 

sentences = []                                                         

next_words = []                                                        

for i in range(0, len(word_list) - maxlen, step):
    sentences.append(word_list[i: i + maxlen])
    next_words.append(word_list[i + maxlen])

print('Number of sequences:', len(sentences))

words = sorted(list(set(word_list)))                                        
print('Unique words:', len(words))
word_indices = dict((word, words.index(word)) for word in words)       

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)      
y = np.zeros((len(sentences), len(words)), dtype=np.bool)              
for i, sentence in enumerate(sentences):                               
    for t, word in enumerate(sentence):                                
        x[i, t, word_indices[word]] = 1                                
    y[i, word_indices[next_words[i]]] = 1 

Number of sequences: 46150
Unique words: 6655
Vectorization...


In [48]:
import nltk
from nltk.corpus import cmudict
nltk.download('cmudict')
d = cmudict.dict()

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\kyle_\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [49]:
# Method to add phonems for each word to 'phonem' dict
def syls(word):
    flag = False
    r_word = word[:]
    while len(r_word) > 0 and flag == False:
        try:
            phonem[word] = d[r_word.lower()]
            flag = True        # if no exception occurs then flag is set indicating phonems added to dict
        except Exception as e:
            r_word = r_word[1:]
    if r_word == '':
        phonem[word] = []

In [50]:
# initializes the phonem dict
phonem = {}
# calls the 'syls' method for each word in the 'words' list
for w in words:
    syls(w)
# phonem

In [51]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(words))))
model.add(layers.Dense(len(words), activation='softmax'))

In [52]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [53]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [54]:
def rhyming_words(word):
    if word != '':
        for key in phonem:
            len_key = len(phonem[key])
            for i in range(len(phonem[word])):
                for j in range(len_key):
                    if word == key or len_key == 0:
                        continue
                    elif len(phonem[word]) == 1:
                        if phonem[word][i][-1] == phonem[key][j][-1]:
                            rhymers.append(key)
                    elif len(phonem[key]) == 2:
                        if phonem[word][i][-2:] == phonem[key][j][-2:]:
                            rhymers.append(key)
                    else:
                        if phonem[word][i][-3:] == phonem[key][j][-3:]:
                            rhymers.append(key)
#     print(len(rhymers))
    
                                           
def syllable_counter(word):
    count = 0
    if ipa.syllable_count(word) == 0:
        count = len(next_word) % 3
        if count == 0:
            count = 1
    else:
        count += ipa.syllable_count(word)
    return count

In [ ]:
import random
import sys

RHYME_CHECK = 5
line_count = 0
quatrain_count = 0
quatrain = False
file = open("poem_output.txt", 'a')

generated_text = ''
count = 0

for epoch in range(1, 11):
    generated_text = ''
    generated_list = []
    print('\nepoch', epoch)
    hist = model.fit(x, y, batch_size=128, epochs=1)                       
    start_index = random.randint(0, len(word_list) - maxlen - 1)         
    generated_list = word_list[start_index: start_index + maxlen]

    for word in generated_list:
        generated_text += word + ' '
    generated_text =  generated_text.strip()
    print('--- Generating with seed: "' + generated_text + '"')
    for num in ipa.syllable_count(generated_text):
        count += num
    for temperature in [0.5, 1.0, 1.2]:                        
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        file.write(generated_text)

        for i in range(400):                                        
            sampled = np.zeros((1, maxlen, len(words)))             
            for t, word in enumerate(generated_list):               
                sampled[0, t, word_indices[word]] = 1. 
                
            if count >= 10:
                sys.stdout.write('\n')
                file.write('\n')
                count = 0    

            if line_count == 2 and count >= RHYME_CHECK:
                preds = model.predict(sampled, verbose=0)[0] * z1
                next_index = sample(preds, temperature)                 
                next_word = words[next_index]
                if count + syllable_counter(next_word) >= 10:
#                         sys.stdout.write('     THIRD LINE')
                    line_count += 1 
                    if quatrain_count == 3:
                        line_count += 1
                        quatrain_count = 0
                else:
                    preds = model.predict(sampled, verbose=0)[0]
                    next_index = sample(preds, temperature)                 
                    next_word = words[next_index]
                    while (count + syllable_counter(next_word)) >= 10:
                        next_index = sample(preds, temperature)                 
                        next_word = words[next_index] 
               
            elif line_count == 3 and count >= RHYME_CHECK:
                preds = model.predict(sampled, verbose=0)[0] * z2
                next_index = sample(preds, temperature)                 
                next_word = words[next_index]
                if (count + syllable_counter(next_word) >= 10):
#                         sys.stdout.write('     FOURTH LINE')
                    line_count += 1
                    quatrain_count += 1
                else:
                    preds = model.predict(sampled, verbose=0)[0]
                    next_index = sample(preds, temperature)                 
                    next_word = words[next_index]
                    while (count + syllable_counter(next_word)) >= 10:
                        next_index = sample(preds, temperature)                 
                        next_word = words[next_index] 

            else:
                preds = model.predict(sampled, verbose=0)[0]
                next_index = sample(preds, temperature)                 
                next_word = words[next_index]

            generated_list.append(next_word)
            generated_list = generated_list[1:]

            sys.stdout.write(' ' + next_word)
            file.write(' ' + next_word)
            syl_num = syllable_counter(next_word)
            count += syl_num

            if count >= 10:
                if line_count == 0:
#                     sys.stdout.write('     FIRST LINE')
                    rhymers = []
                    z1 = np.zeros(len(words)) + 0.0001
#                     print("\nNext Word:", next_word)
                    r_word = next_word[:].lower()
                    r_word = r_word.strip(",.?!:;-'\"_")   
#                     print(r_word)
                    rhyming_words(r_word)
#                     print(rhymers)
                    for rhyme in rhymers:
                        if rhyme in word_indices:
                            z1[word_indices[rhyme]] = 10000.
                    line_count += 1
                    if quatrain_count == 3:
                        line_count += 1

                elif line_count == 1:
#                     sys.stdout.write('     SECOND LINE')
                    rhymers = []
                    z2 = np.zeros(len(words)) + 0.0001
#                     print("\nNext Word:", next_word)
                    r_word = next_word[:].lower()
                    r_word = r_word.strip(",.?!:;-'\"_")   
#                     print(r_word)
                    rhyming_words(r_word)
#                     print(rhymers)
                    for rhyme in rhymers:
                        if rhyme in word_indices:
                            z2[word_indices[rhyme]] = 10000.
                    line_count += 1

                elif line_count == 4:
                    sys.stdout.write('\n')
                    file.write('\n')
                    line_count = 0
file.close()


epoch 1
Epoch 1/1
46150/46150 [==============================] - 81s 2ms/step - loss: 6.8548
--- Generating with seed: "brittle as they are but so"
------ temperature: 0.5
brittle as they are but so love and me
 and love i love not be that thou so but
 not thou art to the same and is in thee
 the of of the prayer and the sky and that

 and thy gainst a and thing my eyes to my
 heart with the be of man and i live this
 doth see i not like the who of my thy
 was i be in thee to the night service

 the world of the most of on and given
 not the yet and the toll of there i i
 your themselves i not love so as and in
 the and into the of his bright of my

 life and the and and still to my love and
 then i love not he love my love i stand

 love not you not at the a sallow of to
 no the and through the of the of the of
 the most of the of of thy and that too
 can i love you do not not and me whereof

 i that thee i love not and not i do
 in me to which for the bonds and of my
 love and not i

 that i will not subject in the body
 of the world of the world i am to that
 i------ temperature: 1.0
sonnet from me to her whose belong slander i have all the robbery

 can write the glory youth's thus the nature's
 weakens pain for mine is month and love's appeals

 but the sea or tis every there then
 every keep my life's happy are one
 long dear that knowing yet winter i pardon
 to thine blood and a little seem for un

 gentle cry look where hours to great more thou
 hast ray art unto the inveteracy air if for
 me let the grey brow with my deserved now
 say her of doth pointed is the need or

 unto my comfort i'll went subject thy
 virtue our part not i can not loved day
 for when i must spirit in that dost i
 loved never show thee found this only a

 conscience and after a husband form find
 to soul beauty all-oblivious help who mouth for bud

 your winds indeed to can his birds set could
 the sin because and run to seemed of this
 seemed nor in your grave should tree walk whom g

 down to long hour non known even in her
 faith the above call no because i any

 him shadows so him love she still my tis
 as thy life i antique on thy glass when
 wait for who none dear or prove praise to his
 alone cold------ temperature: 1.2
then if for my love thou kiss upon doth read him in

 a sea love ground a well lost all to-day and
 fronts end i could in happy poem you sharpened

 of all born the leaues with impotent and
 a meco like life ask thou his body break
 used high more none when how in weary wind
 should worth some cunning in on and brow make

 have forget on corpse is here that voice these
 again as he yet doth but her gored dear
 false we mighty and beauty still your beauty's
 same here only there friend in clear thy year

 thoughts what love's wine hath sovereign us thy wind
 claim with the could oh marriage am deep can
 i mouth it main with such more subjection to kind
 years that am store to lifts deserved turns than

 in his hands i should since day buffet blo

 thine oh for better oh me i shall not

 more thee on disgrace subtle things from earth
 sleep have him no poet constant are at death's
 breathe that wind when thing be wherein it with
 heart new knows from compare with that were grace

epoch 5
Epoch 1/1
46150/46150 [==============================] - 81s 2ms/step - loss: 5.5330
--- Generating with seed: "is lust in action and till"
------ temperature: 0.5
is lust in action and till
 i am at my love and what can say i
 can make us your like like a little heart
 whose to the face that is the but that my
 life the great of this world doth to thee most

 but that i can be so you your i be
 what can i have you too though for it was
 me what is the true of love that which bee
 me with me for one of love and you has

 love for love that love love's love to be then
 more than i think you i have not i in

 your your sweet self so not the name of love
 i know that one of love to see that i
 have thee my love for thee i will not leave
 thee to ha

 is will my fair last beloved in soone

 made as poor a summer's by pen day are
 did men they or all their nay treasure their
 the up time thou how due yours your all our
 best was in the turn to while that that honour

 doth all that which this away and shall up
 wilt thou shalt my go thou rest this thy hope

 un tell thou that thou art canst god's those
 whose could dull in this dull no thing shall there
 know they and live weeds self and that thou rise
 woe rose much love to me thus again more

 is save flesh than love that little write thou
 life me this past dead thee i when her these
 as i pleasure thy these eyes though allow
 but is my and fair but you one thee as

 all one any most may know they ye see
 where they are spread why i all their love and
 i in amore did i was that it she
 thee did long those come has an deep husband

 such were to better grow or the take gift
 love to their on us its spring lose who yet

 then can a love will thine we like mine and
 just those might

In [36]:
hist.history

{'loss': [1.7883135611979672, 1.0]}

In [35]:
hist.history['loss'].append(1.0)